# Описание задачи

Содержание

[1. Предисловие](#1.-Предисловие)


[2. Формулировка задачи и постулаты](#2.-Формулировка-задачи-и-постулаты)

[3. Математическое описание](#3.-Математическое-описание)

[4. Заключение](#4.-Заключение)

# 1. Предисловие

### С чего все началось

На одном собеседовании меня попросили решить "простую задачу":

- Дан набор транспортных стредств (ТС) разной вместимости. 

- Требуется доставить нефть из города-базы различным заказчикам в других городах. 

- В одном городе может быть несколько заказов. 

- Заказ характризуется требуемым объемом, ценой по которой заказчик готов купить нефть и временным окном доставки. 

- В начальный момент времени все ТС находятся на базе. 

- В последний момент времени все ТС также должны быть на базе. 

- Загрузка данного ТС возможна только на базе. 

- Временем на загрузку или разгрузку ТС пренебрегаем.

- Заданы расстояния между городами, стоимость за 1 км пути (для каждого ТС своя), время движения между городами (не зависит от ТС).

- Требуется максимизировать операционную прибыль = выручка от продажи нефти заказчикам минус затраты на доставку. 

Я покопался в интернете и нашел хорошую работу ["Time Constrained Routing and Scheduling"](https://www.researchgate.net/publication/230596630_Time_Constrained_Routing_and_Scheduling). Мне показалось, что данная мне задача похожа на задачу "The Vehicle Routing Problem with Time Windows (VRPTW)", описанную в статье. Поэтому я пытался математически сформулировать "простую задачу в терминах VRPTW.

Промучился 4 дня. Решить, или даже математически правильно сформулировать задачу, не удалось. Послал наработанный CPLEX код. И недоумевал, что это за простая задача.  

В ответ ребята написали, что я почти решил, прислали свой вариант и предложили мне, чтобы я подправил свое решение соотвествующим образом.

Их вариант решения мне что-то "не очень" понравился. Он заключался в следующем:

- Берется одно ТС.

- Выполняет какие-то заказы с учетом максимизации прибыли и тем самым уменьшет спрос.

- Далее иттеративно должны использоваться другие ТС.

- Также "не очень" использование "квантованного" времени и "трюки" с индексами переменных доступные в CPLEX. Что делало задачу не похожей на задачу смешанного целочисленного программирования.

Сразу стало понятно, что свое решение мне подправить не удастся, т.к. наши переменные (decision variables) довольно сильно отличались друг от друга и придется тупо копировать, а это не вариант. И еще момент -- "затачиваться" под решение ребят, квантованное время и запуск одного ТС за другим, мне решительно не хотелось.  

Вобщем я продолжил решать задачу в первичном направалении (VRPTW). Потратил еще 3 дня. Силы закончились. "Уперся в стену".
Поблагодарил ребят за предоставленный второй шанс и отказался от дальнейших собеседованний. 

### Дальнейшие шаги

Мне все таки было интересно решить эту задачу, так чтобы машины ездили по заказчикам, возвращались на базу и опять отправлялись выполнять оставшиеся заказы пока время не истекло. В течение 2-3 месяцев время от времени я обдумывал с какого бока подойти к этой "простой задаче".

Помогла страница [Википедии](https://en.wikipedia.org/wiki/Vehicle_routing_problem). Стало понятно, что мне надо искать задачи класса "Vehicle Routing Problem with Multiple Trips" (VRPMT), когда одно ТС может сделать несколько рейсов.

Обзорная статья Diego Cattaruzza и др. ["Vehicle Routing Problem with Multiple Trips"](https://www.researchgate.net/publication/326722554_Vehicle_routing_problems_with_multiple_trips) сыграла ключевую роль -- помогла определиться с бизнес формулировкой задачи и выбрать переменные для математической формулировки.

_Мой перевод:_

Vehicle Routing Problem with Multiple Trips - Задача маршрутизации ТС с возможностью повторных поездок.

# 2. Формулировка задачи и постулаты

### "Классическая"  задача  VRPMT (Vehicle Routing Problem with Multiple Trips)

Для начала приведу формулировку классической задачи VRPMT, взятую из работы [Diego Cattaruzza и др.](https://www.researchgate.net/publication/326722554_Vehicle_routing_problems_with_multiple_trips)

Пусть $G = (N, A)$ направленный граф, где $N = \{0, 1 , ..., N\}$ набор вершин (городов). Пусть  $A = \{(i, j)\ |\ i, j \in N\}$ набор ребер, соединяющий города. Каждое ребро $(i, j) \in A$ характеризуются временем движения вдоль него $T_{ij}$. В вершине 0 располагается база, в которой находится парк, $V$, одинаковых ТС ограниченной вместимости $Q$.
В начальный момент времени, 0,  все ТС находятся на базе и должны вернуться на нее к моменту времени $T_H$. Вершины $1, ..., N$ можно считать покупателями (закказчиками), которым нужно доставить продукт объема $Q_i\ge0$. В задаче VRPMT требуется определить набор поездок ТС из парка, так чтобы полное время, затраченное на поездки, было мимнимальным, и при этом должны быть выполненны следующие условия:

(1) каждая поездка начинается и заканчивается на базе,

(2) каждый покупатель посещается только 1 раз,

(3) для каждой поездки сумма спроса у покупателей не должна превышать вместимость ТС $Q$, 

(4) полное время всех поездок отдельно взятого ТС не должо превышить $T_H$ - время финального возврата на базу.

Задача VRPMT также может быть дополнена условием:

(5) покупатели должны быть посещены в определенный интервал времени, но теперь это будет задачей "Multiple Trips Vehicle Routing Problem with Time Windows".

С учетом (5), данная формулировка очень похожа на формулировку моей задачи, но есть и отличия. Рассмотрим их подробнее. 

### Отличие моей задачи от классической

Ориентируемся на формулировку из [Предисловия](#1.-Предисловие) и смотрим на отличия.

_1) В классической задаче у всех ТС одинаковая вместимость в моей задаче вместимости разные._

Ввести разные вместимости ТС в классической задаче не проблема. Упомянул  это для проформы.

_2) В классической задаче каждый покупатель посещается только один раз, только одним ТС за весь период. В моем случае покупаль может быть посещен одним или несколькими ТС несколько раз._

Т.е. отдельное ТС может посетить покупателя, исполнить заказ частично, вернуться на базу, забрать новую порцию продукта и ехать к этому покупателю еще раз. 

_3) В моей задаче в одном городе может быть несколько покупателей, а в классической задаче такого нет._

В принципе здесь нет большой проблемы. Каждого покупателя можно рассматривать, как отдельный город и добавить соотвествующие ребра в исходный в граф. Более подробно об этот говориться в [Заключении](#4.-Заключение). Далее я считаю, что в каждом городе находится только один покупатель.

_4) В классичекой задаче сумма спроса у покупателей не превышает вместимость одного ТС, а в моей задаче это возможно._

Это отличие довольно сильное, а если принять во внимание отличие 2), то даже не знаю можно ли отнести мою задачу к задаче "Multiple Trips Vehicle Routing Problem with Time Windows".

### Окончательная формулировка моей задачи

- Дан набор транспортных стредств (ТС) разной вместимости. 

- Требуется доставить продукт из базы, покупателям в разных городах. 

- В одном городе находится 1 покупатель в одном временном окне. Что делать в случае нескольких покупателей в одном городе описано в [Заключении](#4.-Заключение).

- Заказ характризуется требуемым объемом, ценой по которой покупатель готов купить продукт и временным окном доставки.

- В начальный момент времени все ТС находятся на базе. 

- В последний момент времени все ТС также должны быть на базе. 

- Загрузка данного ТС возможна только на базе. 

- Временем на загрузку или разгрузку ТС пренебрегаем.

- Допустимо ожидание разгрузки, т.е. ТС приедет в город раньше, чем требует временное окно. 

- Заданы расстояния между городами, стоимость за 1 км пути (для каждого ТС своя), время движения между городами. время движения не зависит от ТС.

- Величина отдельного заказа может превашать вместимость любого ТС.

- **Важно!!!**

    1. Pаказ может быть выполен не полностью или вообще не выполнен.
    2. Данное ТС может посетить данный город за один рейс только один раз, но не запрещается посещять этот город в другие рейсы.<br>
    _Замечание:_ <br>
    В некоторых статьях вcтречается условие - граф, $G$, должен быть полным (complete). По-моему это не обязательное условие. Однако важно, чтобы не было вершин добраться до которых, и вернуться обратно, можно только через одну вершину. 

- Требуется максимизировать операционную прибыль равную выручке от продаж продукта завычетом затрат на доставку. Также желательно, чтобы ТС возвращались на базу как можно быстрее.

# 3. Математическое описание

### Начальные данные

- $G = (N_0, A)$ - ориентированный граф, где  $N_0 = {0, 1, ..., N}$ набор вершин (покупателей), а $A = \{(i, j) |\ i, j  \in N\}$ набор ребер.
- Известна длинна каждого ребра, $a_{ij}$.
- На каждом ребре задано время движения по нему $T_{ij}$. 
- Считаем, что база находится в вершине 0.
- Покупатели (спрос) находятся в вершинах $N = N_0 \setminus \{0\}$. 
- $V$ - набор различных ТС, причем у каждого своя вместимость $Q_v$ и своя стоимость за единицу пути $C_v$.
- Величина спроса для данной вершины равна $D_i$ единиц, цена за единицу продукта, $P_i$, спрос должен быть удовлетворен во временнном интервале $[T^{(1)}_i, T^{(2)}_i]$,  где $i\in N$.
- Начальный момент времени равен 0 и все ТС находятся на базе.
- Конечный момент времени, к которому все машины должны вернуться на базу, равен $T_H$.

### Переменные

Пусть $R$ - количество рейсов которое может совершить каждое из имеющихся ТС. Это число заранее неизвестно, но может быть оценено. В классической VRPMT задаче его можно положить равным количеству покупателей, $N$. Здесь я его оцениваю, как суммарный спрос на продукт деленный на суммарную вместимость ТС, $R = [\sum D_i/\sum Q_v] + 1$. Таким образом  количество рейсов это набор чисел $\{1..R\}$.

$$
x^{vr}_{ij} = 
\begin{cases}
1, \text{если для рейса }r \in R, \text{ТС } v \in V, \text{проехало по ребру }(i, j) \in A,\\
0, иначе
\end{cases}
$$

$$
y^{vr}_{i} = 
\begin{cases}
1, \text{если для рейса }r \in R, \text{ ТС } v \in V, \text{посетило вершину }i \in N_0,\\
0, иначе
\end{cases}
$$

$t^{vr}_{i} \ge 0 -$  время посещения вершины $i \in N_0$, во время рейса $r \in R, v \in V$.

$t^{vr}_{i} = 0 -$ если ТС не посетило вершину $i$.

$t^{vr}_{H} -$  время возврата ТС на базу по окончании рейса $r \in R$.

$d^{vr}_{i}\ge 0 - $ количество продукта доставленное ТС  $v \in V$, в вершину  $i \in N$.

$L_i \ge 0 -$ недопоставленное количество продукта в вершину $i \in N$. В ограничениях используется, чтобы разрешить неполностью удовлетворять спрос. В целевой фунции используется совместно с множителем $f_p = 100$, чтобы делать недопоставку невыгодной.

#### Целевая функция - максимизировать прибыль

$$
\begin{split}
    \max & \ \sum_{i \in N} \sum_v \sum_r P_i d^{vr}_i \\
         & - \sum_v \sum_r \sum_{i, j} a_{ij} C_v x^{vr}_{ij} \\
         & - \sum_v \sum_r t^{vr}_{E} \\
         & - \sum_{i \in N} f_p P_i L_i 
\end{split}
$$
Здесь первое слагаемое выручка за доставленное сырье. Второе - затраты на доставку. Третье слагаемое заставляет возвращаться на базу, как можно раньше. Четвертое слагаемое - делает недопоставку невыгодной.

#### Ограничения на переменные связанные со временем

$$
\begin{eqnarray*}
    t^{v1}_0 = 0 & \ \ \ & \forall v \in V,  & \ \ \ (1)\\
    t^{vr}_{H} \le t^{vr+1}_0 & \ \ \ & \forall r \in \{1..R-1\}, \forall v \in V & \ \ \  (2)\\
    t^{vr}_0   \le t^{vr+1}_0 & \ \ \ & \forall r \in \{1..R-1\}, \forall v \in V & \ \ \  (3)\\
    t^{vr}_{H} \le T_H & \ \ \ & \forall r \in \{1..R\}, \forall v \in V & \ \ \  (4)\\
    t^{vr}_i + T_{ij} \le t^{vr}_j + T_H (1 - x^{vr}_{ij}) & \ \ \ & 
                \forall r \in \{1..R\}, \forall v \in V, \forall i \in N_0, \forall j \in N & \ \ \  (5)\\
    t^{vr}_i + T_{i0} \le t^{vr}_H + T_H (1 - x^{vr}_{i0}) & \ \ \ & 
                \forall r \in \{1..R\}, \forall v \in V, \forall i \in N_0, \forall j \in N & \ \ \  (6)\\
    T^{(1)}_i y^{vr}_i \le t^{vr}_i \le T^{(2)}_i y^{vr}_i & \ \ \ & \forall r \in \{1..R\}, \forall v \in V, \forall i \in N & \ \ \ (7)\\
\end{eqnarray*}
$$

(1) Начинаем в нулевой момент времени на базе.

(2) Время прибытия на базу меньше, чем время начала следующего рейса.

(3) Время начала рейса меньше, чем время начала ЛЮБОГО следующего рейса. Это казалось бы очевидно, т.к. есть условия (2) и (6). 
   На самом деле это условие запрещает одновременность рейсов через 1. Допустим $R = 4$. Действительно рейс $r=1$ не начнется раньше, чем $r = 2$, но вот $r = 3$, без этого условия, может начаться одновременно с $r=1$. Это условие обнаружено эмперически.

(4) Ограничение на время возврата на базу для любого рейса.

(5) и (6) Ограничения на время отправки из одной вершины и прибытия в другую.

(7) Ограничения на время выполнения заказа.

#### Ограничения на переменные связанные с перемещениями и выполнениями заказов

$$
\begin{eqnarray*}
    \sum_v \sum_r y^{vr}_i \ge 1       & \ \ \ & \forall i \in N & \ \ \  (8)\\
    \sum_v \sum_r d^{vr}_i = D_i - L_i & \ \ \ & \forall i \in N & \ \ \  (9)\\
    \sum_{j \in N_0} x^{vr}_{ij} = \sum_{j \in N_0} x^{vr}_{ji} = y^{vr}_i & \ \ \ & 
            \forall r \in \{1..R\}, \forall v \in V, \forall i \in N & \ \ \ (10)\\
    \sum_{i \in N} d^{vr}_i \le Q_v & \ \ \ & \forall r \in \{1..R\}, \forall v \in V & \ \ \ (11)\\
    d^{vr}_i \le Q_v y^{vr}_ i& \ \ \ & \forall r \in \{1..R\}, \forall v \in V, \forall i \in N & \ \ \ (12)\\
\end{eqnarray*}
$$

(8) Каждая вершина должна быть посещена хотя бы один раз, хотя бы одним ТС. 

_Замечание:_ Это условие введено в задачу из-за переменой $L_i -$ недопоставленное количество продукта. Оно заставляет посещять вершины с заказами. Правда бывают случаи, когда вершина посещается, а разгрузка не производится.

(9) Удовлетворить спрос покупателя в данной вершине, если это возможно.

(10) Поток -  если ТС приехало в вершину, то должно и уехать.

(11) Доставленное сырье за один рейс не превышает вместимость ТС.

(12) Связь между доставленным сырьем и посещением вершины.

# 4. Заключение

Здесь я не претендую на 100% правильность математической формулировки задачи.

В этом проекте мне интересна лишь матемаматическая формулировка. Задача скорее всего не решиться стандартными программами, если покупателей склишком много $\ge 100$. Уже для 7 городов моя программа использующая CBC оптимзатор работала больше 30 минут. Дальше я просто не ждал.

Разаработка алгоритма, или даже реализация одного из известных, выходит за рамки проекта.

### Случай, когда в одном городе больше одного покупателя.

Случай, когда в одном городе больше одного покупателя - можно разбить на два случая:

1. Удовлетворить спрос покупателей в одном городе в одно и тоже окно по времени.
2. Удовлетворить спрос покупателей в одном городе, но в разные окна по времени.

#### 1. Удовлетворить спрос покупателей в одном городе в одно и тоже окно по времени

Этот случай можно свести к задаче описанной выше, если каждого покупателя рассматривать, как отдельный город, т.е разбить город на город1 и город2. В исходный граф следует добавить соответсвующие ребра. Расстояния и время движения вдоль новых ребер взять те же, что были у старого города. Также следует добавить ребра соединяющие два новых города. Длинну этих ребер можно положить равной нулю, а вот время двжения вдоль них равной нулю брать нельзя, т.к. это привродит у циклам из-за того, что ограничения (5), (6) перестают работать. В экпериментах с тестовыми данными проблемы не было, если я брал время движения вдоль этих ребер равным 0.001 от единицы времени используемой в задаче.

#### 2.Удовлетворить спрос покупателей в одном городе, но в разные окна по времени

Аналогично предыдущему случаю - рассматривать каждый заказ, как отдельный город. Добавить сооотвествующие ребра. А для ребер соединяющих покупателей в одном городе и длину ребер и время движения вдоль них можно положить равной 0.  

_Замечание:_

Замена покупателей на города может привести к "взрыву" в количестве переменных. Альтернативный подход -- добавить переменные отслеживающие сколько спроса удовлетворенно у данного покупателя. 